## HW3: Decision Tree, AdaBoost and Random Forest
In hw3, you need to implement decision tree, adaboost and random forest by using only numpy, then train your implemented model by the provided dataset. TA will use the on-hold test label to evaluate your model performance.

Please note that only **NUMPY** can be used to implement your model, you will get no points by simply calling `sklearn.tree.DecisionTreeClassifier`

## Question 1
Gini Index or Entropy is often used for measuring the “best” splitting of the data. Please compute the Entropy and Gini Index of provided data. Please use the formula from [page 5 of hw3 slides](https://docs.google.com/presentation/d/1kIe_-YZdemRMmr_3xDy-l0OS2EcLgDH7Uan14tlU5KE/edit#slide=id.gd542a5ff75_0_15)

In [1]:
# Copy and paste your implementations right here to check your result
# (Of course you can add your classes not written here)
import numpy as np

# sequence: multiple rows
def gini(sequence): 
    s = len(sequence) # seq's length
    cls = np.unique(sequence) # class in seq
    cls_dict = dict() # store class with corresponding amount
    for item in sequence:
        if(item in cls_dict):
            cls_dict[ item ] += 1
        else:
            cls_dict[item] = 1
    g = 1
    for c in cls: # for each class
        p = cls_dict[c]/s # probability of being the class
        g -= pow(p,2) # gini -= p^2
    return g


def entropy(sequence):
    s = len(sequence) # seq's length
    cls = np.unique(sequence) # class in seq
    cls_dict = dict() # store class with corresponding amount
    for item in sequence:
        if(item in cls_dict):
            cls_dict[ item ] += 1
        else:
            cls_dict[item] = 1
    e = 0
    for c in cls: # for each class
        p = cls_dict[c]/s # probability of being the class
        e -= p*np.log2(p) # entropy -= p*log2(p)
    return e

In [2]:
# 1 = class 1,
# 2 = class 2
data = np.array([1,2,1,1,1,1,2,2,1,1,2])

In [3]:
print("Gini of data is ", gini(data))

Gini of data is  0.4628099173553719


In [44]:
print("Entropy of data is ", entropy(data))

Entropy of data is  0.9456603046006401


## Load data
It is a binary classifiation dataset that classify if price is high or not for a cell phone, the label is stored in `price_range` column.

In [11]:
import pandas as pd

train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')
print(train_df.shape)
print(val_df.shape)

train_df.head()

x_train = train_df.drop(labels=["price_range"], axis="columns")
feature_names = x_train.columns.values
x_train = x_train.values
y_train = train_df['price_range'].values

x_test = val_df.drop(labels=["price_range"], axis="columns")
x_test = x_test.values
y_test = train_df['price_range'].values


(1200, 21)
(300, 21)
[1.883e+03 1.000e+00 2.700e+00 1.000e+00 6.000e+00 1.000e+00 3.000e+01
 5.000e-01 9.500e+01 2.000e+00 1.000e+01 5.310e+02 7.910e+02 3.724e+03
 1.600e+01 1.500e+01 2.000e+01 1.000e+00 1.000e+00 1.000e+00] 1
[9.870e+02 1.000e+00 1.900e+00 0.000e+00 4.000e+00 1.000e+00 5.200e+01
 5.000e-01 8.300e+01 3.000e+00 8.000e+00 4.190e+02 7.360e+02 2.757e+03
 1.700e+01 1.200e+01 1.500e+01 1.000e+00 1.000e+00 0.000e+00] 0
[1.306e+03 1.000e+00 2.100e+00 1.000e+00 2.000e+00 1.000e+00 3.300e+01
 4.000e-01 1.740e+02 3.000e+00 9.000e+00 8.670e+02 1.258e+03 2.521e+03
 6.000e+00 5.000e+00 1.600e+01 1.000e+00 0.000e+00 0.000e+00] 0
[1.611e+03 1.000e+00 5.000e-01 1.000e+00 1.100e+01 0.000e+00 3.000e+00
 6.000e-01 9.800e+01 3.000e+00 1.400e+01 3.030e+02 7.140e+02 1.595e+03
 8.000e+00 4.000e+00 9.000e+00 1.000e+00 0.000e+00 1.000e+00] 0
[8.620e+02 1.000e+00 8.000e-01 1.000e+00 3.000e+00 0.000e+00 2.300e+01
 4.000e-01 1.650e+02 2.000e+00 1.500e+01 7.930e+02 1.758e+03 2.780e+02
 1.600e+01 3.

## Question 2
Implement the Decision Tree algorithm (CART, Classification and Regression Trees) and trained the model by the given arguments, and print the accuracy score on the validation data. You should implement two arguments for the Decision Tree algorithm
1. **criterion**: The function to measure the quality of a split. Your model should support `gini` for the Gini impurity and `entropy` for the information gain. 
2. **max_depth**: The maximum depth of the tree. If `max_depth=None`, then nodes are expanded until all leaves are pure. `max_depth=1` equals to split data once


In [18]:
col_type = { 0:'n', 1:'b', 2:'n', 3:'b', 4:'n', 5:'b', 6:'n', 7:'n', 8:'n', 9:'n', 10:'n', # data type of each column(feature)
            11:'n', 12:'n', 13:'n', 14:'n', 15:'n', 16:'n', 17:'b', 18:'b', 19:'b'} #20 = target

# compute gini or entropy using true rows & false rows
def compute(t, f, criterion):
    prob_t = len(t)/len(t)+len(f) 
    print("prob_t: ", prob_t)
    print(t[1])
    if(criterion=='gini'):
        n = gini(t[1])*prob_t + gini(f[1])*(1-prob_t)
    else:
        n = entropy(t[1])*prob_t + entropy(f[1])*(1-prob_t)
    return n

# split by a boolean feature
def tf_bool(x_data, y_data, i):
    t = []
    f = []
    for row, cls in zip(x_data, y_data):
        if (row[i]==1):
            t.append([row,cls])
        else:
            f.append([row,cls])
    t = np.array(t)
    f = np.array(f)
    return t, f

# split by a numeric feature
def tf_num(x_data, y_data, i, v):
    t = []
    f = []
    for row, cls in zip(x_data, y_data):
        if(row[i]>=v):
            t.append([row,cls])
        else:
            f.append([row,cls])
    t = np.array(t)
    f = np.array(f)
    return t, f

# find the best feature & value to split
def split(x_data, y_data, criterion): 
    best = 1
    best_feature_n_val = [] #col, bool/num(type), val(if type=num)

    for i in range(len(col_type)):
        print(i)
        if(col_type=='b'): # if the feature is boolean
            t, f = tf_bool(x_data, y_data, i)
            if(len(t)!=0 and len(f)!=0):
                new = compute(t, f, criterion)
                print(new)
                if (new < best):
                    best = new
                    best_feature_n_val = [i, 'b']
        else: # if the feature is numeric
            val_of_f = []
            for row in x_data:
                val_of_f.append(row[i])
            for v in val_of_f:
                t, f = tf_num(x_data, y_data, i, v)
                print(len(t), len(f), v)
                if(len(t)!=0 and len(f)!=0):
                    new = compute(t, f, criterion)
                    print(new)
                    if (new < best):
                        best = new
                        best_feature_n_val = [i, 'n', v]

    return best, best_feature_n_val
        
            

class d_node():
    def __init__(self, true, false, split_f):
        self.true = true
        self.false = false
        self.split_f = split_f

class l_node():
    def __init__(self, sequence):
        self.cls_pred = dict() # store class with corresponding amount
        for item in sequence:
            if(item in self.cls_pred):
                self.cls_pred[ item ] += 1
            else:
                self.cls_pred[item] = 1

class DecisionTree():
    def __init__(self, criterion='gini', max_depth=None):
        self.max_depth = max_depth
        self.criterion = criterion
        pass

    def fit(self, x_data, y_data):
        
        pass

    def predict(self, x_data):
        pass

    def build(self, x_data, y_data, depth):
        print("in")
        s, s_feature_n_val= split(x_data, y_data, self.criterion) # [i, 'n', v] or [i, 'b']
        
       # if(self.criterion=='gini'):
        #    print(x_data)
         #   cur = gini(x_data)
        #else:
         #   cur = entropy(x_data)

        #if(s==cur):
         #   l = l_node(x_data)
         #   print(l)
         #   print("l_done-----------------")
         #   return l_node(x_data)
        print(s, s_feature_n_val)
        if(s_feature_n_val[1] == 'b'): # col[i] = boolean
            t, f = tf_bool(x_data, y_data, s_feature_n_val[0])
        else:# col[i] = numeric
            t, f = tf_num(x_data, y_data, s_feature_n_val[0], s_feature_n_val[2])

        if(depth<self.max_depth):
            true = self.build(t, self.criterion, depth+1)
            false = self.build(f, self.criterion, depth+1)
        else: 
            l = l_node(x_data)
            print(l)
            print("l_done-----------------")
            return l_node(x_data)

        return d_node(true, false, s_feature_n_val)
        

d = DecisionTree(criterion='gini', max_depth=1)
d.build(x_data = x_test,y_data = y_test, depth = 1)


in
0
[[array([1.883e+03, 1.000e+00, 2.700e+00, 1.000e+00, 6.000e+00, 1.000e+00,
         3.000e+01, 5.000e-01, 9.500e+01, 2.000e+00, 1.000e+01, 5.310e+02,
         7.910e+02, 3.724e+03, 1.600e+01, 1.500e+01, 2.000e+01, 1.000e+00,
         1.000e+00, 1.000e+00])
  1]
 [array([1.987e+03, 1.000e+00, 5.000e-01, 0.000e+00, 3.000e+00, 1.000e+00,
         5.300e+01, 1.000e+00, 1.810e+02, 3.000e+00, 7.000e+00, 1.052e+03,
         1.516e+03, 3.117e+03, 1.800e+01, 5.000e+00, 8.000e+00, 1.000e+00,
         1.000e+00, 1.000e+00])
  1]
 [array([1.994e+03, 0.000e+00, 1.500e+00, 1.000e+00, 2.000e+00, 1.000e+00,
         7.000e+00, 5.000e-01, 1.460e+02, 8.000e+00, 7.000e+00, 4.470e+02,
         1.435e+03, 3.622e+03, 7.000e+00, 0.000e+00, 2.000e+01, 1.000e+00,
         0.000e+00, 1.000e+00])
  1]
 [array([1.972e+03, 0.000e+00, 1.200e+00, 1.000e+00, 0.000e+00, 0.000e+00,
         3.700e+01, 7.000e-01, 1.020e+02, 2.000e+00, 1.000e+00, 7.150e+02,
         1.648e+03, 2.870e+03, 1.600e+01, 0.000e+00, 1.700e

C:\Users\USER\AppData\Local\Temp\ipykernel_18140\626891843.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  t = np.array(t)
C:\Users\USER\AppData\Local\Temp\ipykernel_18140\626891843.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  f = np.array(f)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

### Question 2.1
Using `criterion=gini`, showing the accuracy score of validation data by `max_depth=3` and `max_depth=10`, respectively.


In [8]:
clf_depth3 = DecisionTree(criterion='gini', max_depth=3)
clf_depth10 = DecisionTree(criterion='gini', max_depth=10)

### Question 2.2
Using `max_depth=3`, showing the accuracy score of validation data by `criterion=gini` and `criterion=entropy`, respectively.


In [9]:
clf_gini = DecisionTree(criterion='gini', max_depth=3)
clf_entropy = DecisionTree(criterion='entropy', max_depth=3)

- Note: Your decisition tree scores should over **0.9**. It may suffer from overfitting, if so, you can tune the hyperparameter such as `max_depth`
- Note: You should get the same results when re-building the model with the same arguments,  no need to prune the trees
- Hint: You can use the recursive method to build the nodes


## Question 3
Plot the [feature importance](https://sefiks.com/2020/04/06/feature-importance-in-decision-trees/) of your Decision Tree model. You can get the feature importance by counting the feature used for splitting data.

- You can simply plot the **counts of feature used** for building tree without normalize the importance. Take the figure below as example, outlook feature has been used for splitting for almost 50 times. Therefore, it has the largest importance

![image](https://i2.wp.com/sefiks.com/wp-content/uploads/2020/04/c45-fi-results.jpg?w=481&ssl=1)

## Question 4
implement the AdaBooest algorithm by using the CART you just implemented from question 2 as base learner. You should implement one arguments for the AdaBooest.
1. **n_estimators**: The maximum number of estimators at which boosting is terminated

In [343]:
class AdaBoost():
    def __init__(self, n_estimators):
        pass

    def fit(self, x_data, y_data):
        pass

    def predict(self, x_data):
        pass

### Question 4.1
Show the accuracy score of validation data by `n_estimators=10` and `n_estimators=100`, respectively.


## Question 5
implement the Random Forest algorithm by using the CART you just implemented from question 2. You should implement three arguments for the Random Forest.

1. **n_estimators**: The number of trees in the forest. 
2. **max_features**: The number of random select features to consider when looking for the best split
3. **bootstrap**: Whether bootstrap samples are used when building tree


In [48]:
class RandomForest():
    def __init__(self, n_estimators, max_features, boostrap=True, criterion='gini', max_depth=None):
        pass

    def fit(self, x_data, y_data):
        pass

    def predict(self, x_data):
        pass

### Question 5.1
Using `criterion=gini`, `max_depth=None`, `max_features=sqrt(n_features)`, showing the accuracy score of validation data by `n_estimators=10` and `n_estimators=100`, respectively.


In [12]:
clf_10tree = RandomForest(n_estimators=10, max_features=np.sqrt(x_train.shape[1]))
clf_100tree = RandomForest(n_estimators=100, max_features=np.sqrt(x_train.shape[1]))

### Question 5.2
Using `criterion=gini`, `max_depth=None`, `n_estimators=10`, showing the accuracy score of validation data by `max_features=sqrt(n_features)` and `max_features=n_features`, respectively.


In [13]:
clf_random_features = RandomForest(n_estimators=10, max_features=np.sqrt(x_train.shape[1]))
clf_all_features = RandomForest(n_estimators=10, max_features=x_train.shape[1])

- Note: Use majority votes to get the final prediction, you may get slightly different results when re-building the random forest model

### Question 6. Train and tune your model on a real-world dataset
Try you best to get higher accuracy score of your model. After parameter tuning, you can train your model on the full dataset (train + val).
- Feature engineering
- Hyperparameter tuning
- Implement any other ensemble methods, such as gradient boosting. Please note that you **can not** call any package. Also, only ensemble method can be used. Neural network method is not allowed to used.

In [ ]:
def train_your_model(data):
    ## Define your model and training 
    return

In [4]:
my_model = train_your_model(train_df)

In [ ]:
y_pred = my_model.predict(x_test)

In [39]:
assert y_pred.shape == (500, )

## Supplementary
If you have trouble to implement this homework, TA strongly recommend watching [this video](https://www.youtube.com/watch?v=LDRbO9a6XPU), which explains Decision Tree model clearly. But don't copy code from any resources, try to finish this homework by yourself! 

### DO NOT MODIFY CODE BELOW

In [44]:
import pandas as pd
from sklearn.metrics import accuracy_score
y_test = pd.read_csv('y_test.csv')['price_range'].values

print('Test-set accuarcy score: ', accuracy_score(y_test, y_pred))

Test-set accuarcy score:  0.494


In [ ]:
def discrete_checker(score, thres, clf, name, x_train, y_train, x_test, y_test):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    if accuracy_score(y_test, y_pred) - thres >= 0:
        return score
    else:
        print(f"{name} failed")
        return 0


def patient_checker(score, thres, CLS, kwargs, name,
                    x_train, y_train, x_test, y_test, patient=10):
    while patient > 0:
        patient -= 1
        clf = CLS(**kwargs)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        if accuracy_score(y_test, y_pred) - thres >= 0:
            return score
    print(f"{name} failed")
    print("Considering the randomness, we will check it manually")
    return 0


def load_dataset():
    file_url = "http://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv"
    df = pd.read_csv(
        file_url,
        names=["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
               "Viscera weight", "Shell weight", "Age"]
    )

    df['Target'] = (df["Age"] > 15).astype(int)
    df = df.drop(labels=["Age"], axis="columns")

    train_idx = range(0, len(df), 10)
    test_idx = range(1, len(df), 20)

    train_df = df.iloc[train_idx]
    test_df = df.iloc[test_idx]

    x_train = train_df.drop(labels=["Target"], axis="columns")
    feature_names = x_train.columns.values
    x_train = x_train.values
    y_train = train_df['Target'].values

    x_test = test_df.drop(labels=["Target"], axis="columns")
    x_test = x_test.values
    y_test = test_df['Target'].values
    return x_train, y_train, x_test, y_test, feature_names


score = 0

data = np.array([1, 2])
if abs(gini(data) - 0.5) < 1e-4:
    score += 2.5
else:
    print("gini test failed")

if abs(entropy(data) - 1) < 1e-4:
    score += 2.5
else:
    print("entropy test failed")

x_train, y_train, x_test, y_test, feature_names = load_dataset()

score += discrete_checker(5, 0.9337,
                          DecisionTree(criterion='gini', max_depth=3),
                          "DecisionTree(criterion='gini', max_depth=3)",
                          x_train, y_train, x_test, y_test
                          )

score += discrete_checker(2.5, 0.9036,
                          DecisionTree(criterion='gini', max_depth=10),
                          "DecisionTree(criterion='gini', max_depth=10)",
                          x_train, y_train, x_test, y_test
                          )

score += discrete_checker(2.5, 0.9096,
                          DecisionTree(criterion='entropy', max_depth=3),
                          "DecisionTree(criterion='entropy', max_depth=3)",
                          x_train, y_train, x_test, y_test
                          )

print("*** We will check your result for Question 3 manually *** (5 points)")

score += patient_checker(
    7.5, 0.91, AdaBoost, {"n_estimators": 10},
    "AdaBoost(n_estimators=10)",
    x_train, y_train, x_test, y_test
)

score += patient_checker(
    7.5, 0.87, AdaBoost, {"n_estimators": 100},
    "AdaBoost(n_estimators=100)",
    x_train, y_train, x_test, y_test
)

score += patient_checker(
    5, 0.91, RandomForest,
    {"n_estimators": 10, "max_features": np.sqrt(x_train.shape[1])},
    "RandomForest(n_estimators=10, max_features=sqrt(n_features))",
    x_train, y_train, x_test, y_test
)

score += patient_checker(
    5, 0.91, RandomForest,
    {"n_estimators": 100, "max_features": np.sqrt(x_train.shape[1])},
    "RandomForest(n_estimators=100, max_features=sqrt(n_features))",
    x_train, y_train, x_test, y_test
)

score += patient_checker(
    5, 0.92, RandomForest,
    {"n_estimators": 10, "max_features": x_train.shape[1]},
    "RandomForest(n_estimators=10, max_features=n_features)",
    x_train, y_train, x_test, y_test
)

print("*** We will check your result for Question 6 manually *** (20 points)")
print("Approximate score range:", score, "~", score + 25)
print("*** This score is only for reference ***")
